In [ ]:
import pandas as pd     
import numpy as np 
import glob as glob

Base=[]
csv_files = glob.glob('Data/*.csv')

for filename in csv_files:
    if filename == "Data\\EAM_2020.csv":
        print("ok")
        data = pd.read_csv(filename,sep=",")   
    else:
        data = pd.read_csv(filename,sep=";")
      
    Base.append(data)

In [ ]:
EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20 = Base
#Assign each dataframe a variable of the "Base"
Data_Merge = [EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20]
#Create a list of the dataframe
EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G=\
                0,0,0,0,0,0,0,0,0,0,0,0,0
Data_Gruop = [EAM08_G,EAM09_G,EAM10_G,EAM11_G,EAM12_G,EAM13_G,EAM14_G, \
            EAM15_G,EAM16_G,EAM17_G,EAM18_G,EAM19_G,EAM20_G]

In [ ]:
#PARA CAMBIAR UN LABEL EN UNA UBICACIÓN ESPECIFICA NO ESTANDARIZADO
def change_name(Data_Merge, Number, change):
    for df in Data_Merge:
        if len(df.columns) > Number: #Verificar si el dataframe tiene al menos 4 columnas
            df.columns.values[Number] = change #Me cambia el name de el label 
    return Data_Merge

In [ ]:
#Para estandairzar los labels
def change_label(Data_Merge):
    for df in Data_Merge:
        columnas = df.columns.tolist() #Crea una lista de columnas
        nuevas_columnas = [columna.upper() for columna in columnas] # coloca en mayusculas cada columna de lista
        df.columns = nuevas_columnas #Asigna el nuevo nombre
    return Data_Merge

In [ ]:
#Función para saber si una columna esta presente en un dataframe
def Exis(Data_Merge, item):
    for i, df in enumerate(Data_Merge):
        if item not in df.columns:
            columna_ausente = True
            print(f"El item no está presente en el DataFrame {i}.")
        else:
            columna_ausente = False
            print(f"El item  está presente en el DataFrame {i}.")
    return columna_ausente

In [ ]:
#Función para saber si existen datos faltantes en la columna
def exis_colum(Data_Merge,colum):
    missing_col=0
    for i in Data_Merge:
        missing_col += i[colum].isnull().sum()
    return missing_col

In [ ]:
#Función para ordenar, por una columna 
def Sort(Column,Data_Merge):
    for i, d in enumerate(Data_Merge):
        sorted_df = d.sort_values(by=Column)
        Data_Merge[i] = sorted_df 
    return Data_Merge

In [ ]:
def calculate_avg_or_sum(Column, SUMATION, UBACATION, Data_Gruop, Data_Merge, operation):
    for i, df in enumerate(Data_Gruop):
        for index in df.index:
            if index in Data_Merge[i][Column].unique():
                total = Data_Merge[i][Data_Merge[i][Column] == index][SUMATION]
                if operation == 'mean':
                    total=total.mean()
                    df.loc[index, UBACATION] = total
                elif operation == 'sum':
                    total=total.sum()
                    df.loc[index, UBACATION] = total
                elif operation == 'count':
                    total=total.value_counts()           
                    df.loc[index, UBACATION] = int(total)           
                elif operation == 'coe_sim':
                    total=total.skew()  #Da como resultado el coeficiente de asimetria de fisher
                    df.loc[index, UBACATION] = total
                elif operation == "med":
                    total=total.median()
                    df.loc[index, UBACATION] = total
                elif operation =="rango":
                    total = total.max() - total.min()
                    df.loc[index, UBACATION] = total
    return Data_Gruop

In [ ]:
def sum_of_rows(row,Colum1,Colum2):
    return row[Colum1] + row[Colum2]

In [ ]:
def IHH(Column, Divisor, Numerator, Ubac, Data_Gruop, Data_Merge):
  for i, df in enumerate(Data_Gruop):
    for index in df.index:
      if index in Data_Merge[i][Column].unique():               
        Numerator = Data_Merge[i].set_index(Column)[Divisor]
        Denominator = df["SUM-WORKERS"]
        Total = (Numerator / Denominator) * 100
        Data_Merge[i].loc[Data_Merge[i][Column], Ubac] = Total
  return Data_Merge

In [ ]:
Data_Merge = change_name(Data_Merge,3,"CIIU")

In [ ]:
Data_Merge = change_label(Data_Merge)

In [ ]:
exis_colum(Data_Merge,"CIIU")
exis_colum(Data_Merge,"C4R4C9T")
exis_colum(Data_Merge,"C4R4C10T")

In [ ]:
Data_Merge=Sort("CIIU",Data_Merge)

In [ ]:
#Para crear un data frame agrupado de una base de datos
for i in range (len(Data_Merge)):
    index=Data_Merge[i]["CIIU"].drop_duplicates().sort_values()
    Data_Gruop[i]= pd.DataFrame(index=index,columns=["SUM-WORKERS_M","SUM-WORKERS_F","SUM-WORKERS","MED-WAGE","IHH","Count_CIIU",'Simetria',"rango"])


In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C4R4C9T","SUM-WORKERS_F",Data_Gruop,Data_Merge,"sum")
Data_Gruop=calculate_avg_or_sum("CIIU","C4R4C10T","SUM-WORKERS_M",Data_Gruop,Data_Merge,"sum")
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","MED-WAGE",Data_Gruop,Data_Merge,"med")

In [ ]:
for df  in Data_Gruop:
    df["SUM-WORKERS"] = df.apply(lambda row: sum_of_rows(row,"SUM-WORKERS_M","SUM-WORKERS_F"), axis=1)

In [ ]:
for df  in Data_Merge:
    df["WORKERS"] = df.apply(lambda row: sum_of_rows(row,"C4R4C10T","C4R4C9T"), axis=1)
    df["IHH_ONE"] = None

In [ ]:
Data_Gruop[1]

In [ ]:
Data_Merge=IHH("CIIU","WORKERS","SUM-WORKERS","IHH_ONE",Data_Gruop,Data_Merge) #Aquí estoy sacando el ihh

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","IHH_ONE","IHH",Data_Gruop,Data_Merge,"sum") ##Sumo el IHH individual

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","CIIU","Count_CIIU",Data_Gruop,Data_Merge,"count")   ## Cuanto la cantidad de empresas que hay ´por CIIU 

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","Simetria",Data_Gruop,Data_Merge,"coe_sim")  #Miro el coeficiente de simetria

In [ ]:
Data_Gruop=calculate_avg_or_sum("CIIU","C3R2C3","rango",Data_Gruop,Data_Merge,"rango")

In [ ]:
for i, df in enumerate(Data_Gruop):
    df["IHH_cuadrado"] = df["IHH"] ** 2

    conditions = [
        df["IHH_cuadrado"] <= 1000,
        (1000 <= df["IHH_cuadrado"]) & (df["IHH_cuadrado"] < 1800),
        df["IHH_cuadrado"] >= 1800
    ]
    values = [1, 2, 3]

    df["IHH_resultado"] = np.select(conditions, values, default=np.nan)

    #Hay un problema y es que son 3 .-.

In [ ]:
exis_colum(Data_Gruop,"IHH")

In [ ]:
Data_Gruop[0]

In [ ]:
#Promedios Ponderados  -- para calcular 
#Para el salario se saca la mediana pagada por cada sector 
# Para saber si el promedio es un buen estadistico se realiza un examen de simetria
#Datos de estilizados -- Indicador: 

Se comienza a trabajar con los datos sacados por hechos estilizados

Se imprime una grafica de barras del IHH por sector para todos los años

In [ ]:
def count_IHH(Grupo_datos, df,SUM):
    for i, data_frame in enumerate(Grupo_datos):
        conteo = data_frame[SUM].value_counts()
        for valor in conteo.index:
            conteo_valor = conteo.loc[valor]
            df.at[df.index[i], valor] = conteo_valor
    return df

In [ ]:
Index=[]
for i in range(len(Data_Gruop)):
    w = 2008 + i 
    Index.append(w)
Columns = [1, 2, 3]
df = pd.DataFrame(index=Index, columns=Columns)
df

In [ ]:
df=count_IHH(Data_Gruop,df,"IHH_resultado")

In [ ]:
# Crear una figura para el gráfico
plt.figure()

# Graficar los resultados de IHH_resultado con un gráfico de barras
df.plot(kind='bar', width=0.8)

# Etiquetas y título del gráfico
plt.xlabel("Años")
plt.ylabel("Cantidad de Sectores")
plt.title("Resultados de IHH por Índice")

# Mostrar el gráfico
plt.show()

Se imprime una grafica con el promedio del salario medio pagado por sectores con poder y sin poder.

In [ ]:
Prom =[]
for i in range(len(Data_Gruop)):
    w = 2008 + i 
    Prom.append(w)
X=['Salario_Medio']
Promedio_years= pd.DataFrame(index=Index, columns=X)

#Creamos un dataframe donde agrupa la mediana de cada CIIU para cada año

In [ ]:
year = 2008 
for df in Data_Gruop:
    median_wage = df["MED-WAGE"].median()
    Promedio_years.at[year, 'Salario_Medio'] = median_wage
    year += 1

In [ ]:
# Leer el archivo de Excel
IPC_doc = pd.read_excel("Data\\IPC.xlsx", header=0)

# Crear un DataFrame IPC a partir de los datos leídos
IPC = pd.DataFrame({'Año': IPC_doc.iloc[:, 0], 'IPC': IPC_doc.iloc[:, 1]})

Index = []
for i in range(len(Data_Gruop)):
    w = 2008 + i 
    Index.append(w)

# Establecer el índice del DataFrame IPC usando la lista Index
IPC.set_index('Año', inplace=True)

# Reemplazar el índice existente con la lista Index
IPC.index = Index

# Imprimir el DataFrame resultante
print(IPC)



In [ ]:
Promedio_years

In [ ]:
Promedio_years["InflaciónIPC"] = IPC["IPC"]
Promedio_years['REAL_WAGE'] = (Promedio_years["Salario_Medio"]  / Promedio_years["InflaciónIPC"]) *100
Promedio_years

In [ ]:
import matplotlib.pyplot as plt

# Supongamos que 'df' es tu DataFrame con una columna "Datos" y un índice deseado en el eje X

# Obtén los valores del índice y los datos de las columnas
x_values = Promedio_years.index
y_values_1 = Promedio_years['REAL_WAGE']
y_values_2 = Promedio_years['Salario_Medio']

# Crea la figura y los ejes
fig, ax1 = plt.subplots()

# Crea un segundo eje Y que comparta el mismo eje X
ax2 = ax1.twinx()

# Trazar datos en el primer eje Y
ax1.plot(x_values, y_values_1, marker='o', linestyle='-', color='b', label='REAL_WAGE')
ax1.set_xlabel('Años')
ax1.set_ylabel('Mediana del Salario REAL', color='b')

# Trazar datos en el segundo eje Y
ax2.plot(x_values, y_values_2, marker='s', linestyle='--', color='r', label='Salario_Medio')
ax2.set_ylabel('Salario Medio', color='r')

# Agregar leyendas para las líneas
ax1.legend(loc='upper left', bbox_to_anchor=(0.05, 0.95))
ax2.legend(loc='upper left', bbox_to_anchor=(0.05, 0.87))

# Título del gráfico
plt.title('Mediana del Salario y Salario Medio Real')

# Muestra el gráfico
plt.show()

In [ ]:

import matplotlib.pyplot as plt

# Supongamos que 'df' es tu DataFrame con una columna "Datos" y un índice deseado en el eje X

# Obtén los valores del índice y los datos de la columna
x_values = Promedio_years.index
y_values= Promedio_years['Salario_Medio']


# Crea el gráfico de línea con puntos
plt.plot(x_values, y_values, marker='o', linestyle='-')

# Etiqueta de los ejes
plt.xlabel('Años')
plt.ylabel('Mediana del Salario')

# Título del gráfico
plt.title('Mediana del Salario pagado por año')

# Muestra el gráfico
plt.show()

In [ ]:

# # Crear una figura para el gráfico
# plt.figure()

# # Graficar los resultados de IHH_resultado con un gráfico de barras
# df.plot(kind='line', marker='o')

# # Etiquetas y título del gráfico
# plt.xlabel("Años")
# plt.ylabel("Cantidad de Sectores")
# plt.title("Resultados de IHH por Índice")

# # Mostrar el gráfico
# plt.show()